In [1]:
#%load_ext lab_black
import pandas as pd
import numpy as np
import scipy
import statsmodels.api as sm
from statsmodels.sandbox.regression import gmm
import matplotlib.pyplot as plt
import seaborn as sns

In a simulation exercise consider the regression $y_i = \alpha + \beta_1 x_{1i} + \beta_2 x_ {2i} + u_i$ with $Cov(u_i, x_{1i}) = 0$ but $Cov(u_i,x_{2i})\neq 0$. This means that one of the regressors is endogenous

#### To DO:

- [x] (a) OLS
- [x] (b) Linear IV
- [ ] (c) 2SLS, (efficient) GMM
  - [x] 2SLS
  - [ ] GMM
  - [ ] efficient GMM
- [x] (d) Weak IV
- [ ] (e) Summary & discussion
  - [x] violin plot to compare different estimators
  - [x] plots to compare different correlation/ weakness of instrument

In [2]:
# set this to true if you only want to test whether function work. set to false to properly conduct the experiment with 1000 repitions.
test_run = False

In [3]:
params = {
    'alpha': 1.1,
    'beta_1': 0.3,
    'beta_2': 0.8,
    'cov_x2_u': 0.6,
    'cov_x2_z_strong': 0.79,
    'cov_x2_z_moderate': 0.6,
    'cov_x2_z_weak': 0.4,
    'cov_x2_z_poor': 0.25,
}

In [4]:
# set ordered of variables in covariance list
params['var_list'] = ['x1', 'x2', 'z_strong', 'z_moderate', 'z_weak', 'z_poor', 'u']

# with 0 correlation between z & u, 
# and specified correlation > 0: among x2 & u and x2 & z and x2 & z_weak, 
# correlation among instruments are just plausible values that make it easy for the distribution to hold true
params['cov'] = np.array(
    [
        [1, 0, 0, 0, 0, 0, 0],
        [0, 1, params['cov_x2_z_strong'], params['cov_x2_z_moderate'], params['cov_x2_z_weak'], params['cov_x2_z_poor'], params['cov_x2_u']],
        [0, params['cov_x2_z_strong'], 1, 0.5, 0.5, 0.5, 0],
        [0, params['cov_x2_z_moderate'], 0.5, 1, 0.5, 0.5, 0],
        [0, params['cov_x2_z_weak'], 0.5, 0.5, 1, 0.5, 0],
        [0, params['cov_x2_z_poor'], 0.5, 0.5, 0.5, 1, 0],
        [0, params['cov_x2_u'], 0, 0, 0, 0, 1],
    ]
)
            
pd.DataFrame(params['cov'], columns = params['var_list'], index = params['var_list'])

successfully found positive definite matrix at iteration 1


,x1,x2,z_strong,z_moderate,z_weak,z_poor,u
x1,1.0,0.00,0.00,0.00,0.00,0.00,0.0
x2,0.0,1.00,0.79,0.60,0.40,0.25,0.6
z_strong,0.0,0.79,1.00,0.74,0.53,0.40,0.0
z_moderate,0.0,0.60,0.74,1.00,0.18,0.18,0.0
z_weak,0.0,0.40,0.53,0.18,1.00,0.73,0.0
z_poor,0.0,0.25,0.40,0.18,0.73,1.00,0.0
u,0.0,0.60,0.00,0.00,0.00,0.00,1.0


In [5]:
def simulate(params: dict, n: int) -> pd.DataFrame:
    """Simulate data."""
    df = pd.DataFrame({'x0': np.ones(n)})
    # create multiple variables based on the covariance matrix and store them into nested np array
    multi_vars = np.random.multivariate_normal(mean=len(params['cov'])*[0], cov=params['cov'], size=n)
    # extract data for variables
    for i, name in enumerate(params['var_list']):
        df[name] = multi_vars[:,i]
    
    df['y'] = (
        params['alpha']
        + params['beta_1'] * df['x1']
        + params['beta_2'] * df['x2']
        + df['u']
    )
    return df

In [6]:
df = simulate(params, 10000)

In [7]:
df.corr().round(3)

,x0,x1,x2,z_strong,z_moderate,z_weak,z_poor,u,y
x0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x1,NaN,1.000,0.000,0.004,0.008,0.021,0.001,-0.009,0.177
x2,NaN,0.000,1.000,0.788,0.593,0.391,0.250,0.610,0.857
z_strong,NaN,0.004,0.788,1.000,0.739,0.523,0.397,0.009,0.388
z_moderate,NaN,0.008,0.593,0.739,1.000,0.168,0.179,-0.000,0.289
z_weak,NaN,0.021,0.391,0.523,0.168,1.000,0.727,0.002,0.195
z_poor,NaN,0.001,0.250,0.397,0.179,0.727,1.000,0.012,0.129
u,NaN,-0.009,0.610,0.009,-0.000,0.002,0.012,1.000,0.904
y,NaN,0.177,0.857,0.388,0.289,0.195,0.129,0.904,1.000


In [8]:
df.describe().round(3)

,x0,x1,x2,z_strong,z_moderate,z_weak,z_poor,u,y
count,10000.0,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,1.0,0.005,-0.001,-0.001,0.002,0.002,0.005,-0.001,1.100
std,0.0,1.001,1.001,0.990,1.000,1.000,1.004,1.005,1.649
min,1.0,-3.613,-3.962,-3.934,-3.531,-3.745,-3.759,-3.667,-4.610
25%,1.0,-0.678,-0.677,-0.665,-0.675,-0.665,-0.666,-0.689,-0.037
50%,1.0,-0.004,0.004,-0.004,0.001,-0.009,0.002,-0.010,1.113
75%,1.0,0.681,0.681,0.672,0.669,0.664,0.672,0.684,2.222
max,1.0,4.102,3.627,4.286,3.593,4.041,4.098,3.767,7.083


(a) Estimate the generated data with OLS, replicate the experiment 1000 times, and observe the empirical distribution of the parameter models. Use sample sizes 200, 500, 1000.

In [11]:
def estimate_model(params: dict, n_obs: list, n_repetitions: int, model: str ='OLS', IV: str = "z") -> dict:
    output = {}
    for n in n_obs:
        params_est = []
        for i in range(n_repetitions):
            np.random.seed(100 + i)
            df = simulate(params, n)
            if model == 'OLS':
                mdl = sm.OLS(df['y'], df[['x0', 'x1', 'x2']]).fit()
                mdl_params = mdl.params
            elif model == 'linearIV':
                mdl = gmm.LinearIVGMM(
                    endog=df['y'],
                    exog=df[['x0', 'x1', 'x2']],
                    instrument=df[['x0', 'x1', IV]],
                ).fit()
                mdl_params = mdl.params
                mdl_params = pd.Series(mdl_params, index=['x0', 'x1', 'x2'])
            elif model == '2SLS':
                mdl = gmm.IV2SLS(
                    endog=df['y'],
                    exog=df[['x0', 'x1', 'x2']],
                    instrument=df[['x0', 'x1', IV]],
                ).fit()
                mdl_params = mdl.params
            elif model == 'GMM':
                # To-Do implement this model
                mdl = gmm
            elif model == 'Efficient_GMM':
                # To-Do implement this model
                mdl = gmm
            else:
                raise ValueError(f'Model {model} not implemented, select OLS or IV')
            params_est.append(mdl_params)
        output[n] = pd.concat(params_est, axis=1)
    return output

In [12]:
def plot_params_dist(estimates: dict, n_obs: list, note: str, params: dict):
    fig, ax = plt.subplots(3, 3, figsize=(13, 10))
    ax = ax.flatten()
    i = 0
    for n in n_obs:
        for col, name, color, limits in zip(
            estimates[n].index,
            ['alpha', 'beta_1', 'beta_2'],  # ['x0', 'x1', 'x2'],
            ['orange', 'red', 'blue'],
            [(0.7, 1.5), (0, 0.7), (0.5, 1.7)],
        ):
            ax[i].vlines(
                x=params[name],
                ymin=0,
                ymax=100,
                color='k',
                alpha=0.7,
                linestyles='dotted',
                label='True',
            )
            estimates[n].T[col].hist(
                ax=ax[i], alpha=0.6, color=color, grid=False, bins=50
            )
            ax[i].set_title(rf'$\{name}$, n={n}')
            ax[i].set_xlim(limits[0], limits[1])
            ax[i].set_ylim(0, 100)
            if i == 8:
                plt.legend()
            i += 1
        plt.suptitle(f'Distribution of parameter for {note} estimates', fontsize=16)

In [13]:
# define all models
models = {
   'ols': {
       'label': 'OLS, no IV',
       'model': 'OLS',
       'IV': None
   },
#    'gmm': {
#        'label': 'GMM',
#        'model': 'GMM',
#        'IV': 'z_strong'
#    },
#    'eff_gmm': {
#        'label': 'Efficient GMM',
#        'model': 'Efficient_GMM',
#        'IV': 'z_strong'
#    },
   'linear_iv_strong': {
       'label': 'Strong IV, linear',
       'model': 'linearIV',
       'IV': 'z_strong'
   },
   'linear_iv_moderate': {
       'label': 'Moderate IV, linear',
       'model': 'linearIV',
       'IV': 'z_moderate'
   },
   'linear_iv_weak': {
       'label': 'Weak IV, linear',
       'model': 'linearIV',
       'IV': 'z_weak'
   },
   'linear_iv_poor': {
       'label': 'Poor IV, linear',
       'model': 'linearIV',
       'IV': 'z_poor'
   },
   '2sls_iv_strong': {
       'label': 'Strong IV, 2SLS',
       'model': '2SLS',
       'IV': 'z_strong'
   },
   '2sls_iv_moderate': {
       'label': 'Moderate IV, 2SLS',
       'model': '2SLS',
       'IV': 'z_moderate'
   },
   '2sls_iv_weak': {
       'label': 'Weak IV, 2SLS',
       'model': '2SLS',
       'IV': 'z_weak'
   },
   '2sls_iv_poor': {
       'label': 'Poor IV, 2SLS',
       'model': '2SLS',
       'IV': 'z_poor'
   },
}

In [14]:
def estimate_and_plot_model(model_key:str, params: dict=params, models:dict=models, test_run: bool=False):
    if not (model_key in models.keys()):
        raise TypeError("model_key is not defined in models")
    n_repetitions= 50 if test_run else 1000
    n_obs = [200, 500, 1000]
    models[model_key]['estimates'] = estimate_model(
        params=params, n_obs=n_obs, n_repetitions=n_repetitions, model=models[model_key]['model'], IV=models[model_key]['IV']
    )
    plot_params_dist(estimates=models[model_key]['estimates'], n_obs=n_obs, note=models[model_key]['label'], params=params)

In [15]:
# # estimate all models. This might take 10-15 minutes.
# n_repetitions= 50 if test_run else 1000
# for key in models.keys():
#     n_obs = [200, 500, 1000]
#     models[key]['estimates'] = estimate_model(
#         params=params, n_obs=n_obs, n_repetitions=n_repetitions, model=models[key]['model'], IV=models[key]['IV']
#     )
#     print('Completed estimating', models[key]['label'], '.', list(models.keys()).index(key)+1, 'out of', len(models.keys()) )

In [16]:
estimate_and_plot_model(model_key='ols', params=params, models=models, test_run=test_run)

(b) Construct valid instruments for x2, use the linear IV estimator, and repeat the experiment as in the previous point.

In [17]:
estimate_and_plot_model(model_key='linear_iv_strong', params=params, models=models, test_run=test_run)

(c) Construct several valid instruments for x2 and estimate with 2SLS, GMM and efficient GMM. Repeat the experiment.

In [18]:
estimate_and_plot_model(model_key='2sls_iv_strong', params=params, models=models, test_run=test_run)

In [19]:
# To-Do implement model.
# estimate_and_plot_model(model_key='gmm', params=params, models=models, test_run=test_run)

In [20]:
# To-Do implement model.
# estimate_and_plot_model(model_key='eff_gmm', params=params, models=models, test_run=test_run)

(d) Construct a weak instrument for x2, where the valid instrument is only very weakly correlated with x2 (in population). Repeat the experiment for the IV and 2SLS estimators.

In [21]:
estimate_and_plot_model(model_key='linear_iv_weak', params=params, models=models, test_run=test_run)

In [22]:
estimate_and_plot_model(model_key='2sls_iv_weak', params=params, models=models, test_run=test_run)

(e) Feel free to play around with different scenarios, distributions for variables, strength of correlation between the valid instrument(s) and x2. Summarize and discuss your results.

In [23]:
# estimate all models that are defined in models and haven been estimated yet in the previous tasks
for key in models.keys():
    if not ('estimates' in models[key].keys()):
        estimate_and_plot_model(model_key=key, params=params, models=models, test_run=test_run)
        print('Completed estimating', models[key]['label'])

Completed estimating Moderate IV, linear
Completed estimating Poor IV, linear
Completed estimating Moderate IV, 2SLS
Completed estimating Poor IV, 2SLS


In [24]:
# Change data format for violinplot 

# stack models estimates
model_estimates_stacked = {}
for key in models.keys():
    model_estimates_stacked[key] = pd.concat(models[key]['estimates']).stack()

# concat different models into single dataframe
df_ = pd.concat(
    [model_estimates_stacked[key] for key in model_estimates_stacked.keys()],
    keys=[models[key]['label'] for key in model_estimates_stacked.keys()],
).reset_index()

# rename columns
df_ = df_.replace({
    'x0': 'alpha','x1': 'beta_1', 'x2': 'beta_2', 
    'z_strong': 'beta_2', 'z_moderate': 'beta_2', 'z_weak': 'beta_2', 'z_poor': 'beta_2'
    })
df_ = df_.rename(columns={0: 'estimate', 'level_0': 'method'})

In [25]:
fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(22, 22))
ax = ax.flatten()
for i, n_obs in enumerate([200, 500, 1000]):
    for j in range(2):
        axIndex = i*2+j
        column = axIndex % 2 # set whether to obtain results for beta_2 (case == 0) or for beta_1 (case == 1)
        parameter = 'beta_2' if column == 0 else 'beta_1'
        parameter_formatted = 'beta_2' if column == 0 else 'beta_1'
        
        sns.violinplot(
            x='method',
            y='estimate',
            by='level_2',
            data=df_[(df_.level_1 == n_obs) & (df_.level_2 == parameter_formatted)],
            ax=ax[i*2+j],
        )
        ax[axIndex].hlines(
            xmin=-0.5,
            xmax=len(df_['method'].unique()) - 0.5,
            y=params[parameter],
            color='k',
            linestyle='dashed',
            alpha=0.6,
        )
        
        # fix ylim to see difference between changes of n_obs
        ax[axIndex].set_ylim(
            min(df_[df_.level_2 == parameter_formatted]['estimate'])-0.1, 
            max(df_[df_.level_2 == parameter_formatted]['estimate'])+0.1
            )

        ax[axIndex].set_title(rf'Estimates for $\{parameter}$, N={n_obs}')

plt.tight_layout()

- weak instruments reduce the precision of the estimates
- do weak instruments bias results for other vars aswell?
    - yes, in small sample sizes weak instruments may distort the result of the exogenous variable aswell. However in larger sample sizes this problem disappears 
- OLS is biased when IV is not implemented 
